In [1]:
# Preprocessing Data for suggroate model 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import seaborn as sns
import sklearn
import imblearn
import matplotlib.pyplot as plt
import time
import sklearn.metrics as m
import xgboost as xgb
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
df1 = pd.read_csv('../datasets/CICIDS2018/02-14-2018.csv') # datasets\CICIDS2018\02-14-2018.csv # Benign、SSH-Bruteforce、FTP-BruteForce
df1 = df1.drop_duplicates(keep='first')
df2 = pd.read_csv('../datasets/CICIDS2018/02-15-2018.csv') # datasets\CICIDS2018\02-15-2018.csv # Benign、DoS attacks-GoldenEye、DoS attacks-Slowloris
df2 = df2.drop_duplicates(keep='first')
df3 = pd.read_csv('../datasets/CICIDS2018/02-16-2018.csv') # datasets\CICIDS2018\02-16-2018.csv # Benign、DoS attacks-Hulk、DoS attacks-SlowHTTPTest
df3 = df3.drop_duplicates(keep='first')
# df4 = pd.read_csv('../datasets/CICIDS2018/02-20-2018.csv') # datasets\CICIDS2018\02-20-2018.csv # Benign 、DDoS attacks-LOIC-HTTP
# df4 = df4.drop_duplicates(keep='first')
df5 = pd.read_csv('../datasets/CICIDS2018/02-21-2018.csv') # datasets\CICIDS2018\02-21-2018.csv # DDOS attack-HOIC、Benign、DDOS attack-LOIC-UDP
df5 = df5.drop_duplicates(keep='first')
df6 = pd.read_csv('../datasets/CICIDS2018/02-22-2018.csv') # datasets\CICIDS2018\02-22-2018.csv # Benign Brute Force -Web 、Brute Force -XSS、SQL Injection
df6 = df6.drop_duplicates(keep='first')
df7 = pd.read_csv('../datasets/CICIDS2018/02-23-2018.csv') # datasets\CICIDS2018\02-23-2018.csv # Benign、Brute Force -Web、Brute Force -XSS、SQL Injection
df7 = df7.drop_duplicates(keep='first')
df8 = pd.read_csv('../datasets/CICIDS2018/02-28-2018.csv') # datasets\CICIDS2018\02-28-2018.csv # Benign、Infilteration
df8 = df8.drop_duplicates(keep='first')
df9 = pd.read_csv('../datasets/CICIDS2018/03-01-2018.csv') # datasets\CICIDS2018\03-01-2018.csv
df9 = df9.drop_duplicates(keep='first')
df10 = pd.read_csv('../datasets/CICIDS2018/03-02-2018.csv') # datasets\CICIDS2018\03-02-2018.csv
df10 = df10.drop_duplicates(keep='first')

df3 = df3[df3['Label'] != 'Label']
df6 = df6[df6['Label'] != 'Benign']
df7 = df7[df7['Label'] != 'Benign']
df8 = df8[df8['Label'] != 'Benign' ]
df8 = df8[df8['Label'] != 'Label' ]
df9 = df9[df9['Label'] != 'Benign']
df9 = df9[df9['Label'] != 'Label']
df10 = df10[df10['Label'] != 'Benign']



In [3]:
df = pd.concat([df1,df2,df3,df5,df6,df7,df8,df9,df10])
df

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,Benign
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356594,0,0,02/03/2018 03:53:22,9487,10,0,0,0.0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bot
356709,51395,6,02/03/2018 03:54:05,1030059,4,2,0,0.0,0,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bot
356768,51397,6,02/03/2018 03:54:11,1029998,4,2,0,0.0,0,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bot
357370,51396,6,02/03/2018 03:54:08,1030017,4,2,0,0.0,0,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bot


In [4]:
df.value_counts('Label')

Label
Benign                      2468266
DDOS attack-HOIC             668461
DoS attacks-Hulk             434883
Bot                          282310
Infilteration                161898
SSH-Bruteforce               117322
DoS attacks-GoldenEye         41455
FTP-BruteForce                39352
DoS attacks-SlowHTTPTest      21092
DoS attacks-Slowloris         10285
DDOS attack-LOIC-UDP           1730
Brute Force -Web                611
Brute Force -XSS                230
SQL Injection                    87
Name: count, dtype: int64

In [5]:
# add column to the dataframe
# if the label is benign, then it is benign, otherwise malicious
df['binary_classification'] = df.apply(lambda row: 'Benign' if row.Label == 'Benign' else 'Malicious', axis=1)


In [6]:
# drop nan values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,binary_classification
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,...,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign,Benign
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,...,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign,Benign
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,...,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign,Benign
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,...,0.0,0.0,0,0,0.0,0.0,0,0,Benign,Benign
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,...,0.0,0.0,0,0,0.0,0.0,0,0,Benign,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356594,0,0,02/03/2018 03:53:22,9487,10,0,0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bot,Malicious
356709,51395,6,02/03/2018 03:54:05,1030059,4,2,0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bot,Malicious
356768,51397,6,02/03/2018 03:54:11,1029998,4,2,0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bot,Malicious
357370,51396,6,02/03/2018 03:54:08,1030017,4,2,0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bot,Malicious


In [7]:
df['binary_classification'].value_counts()

binary_classification
Benign       2456429
Malicious    1778517
Name: count, dtype: int64

In [8]:
# Removing whitespaces in column names.
col_names = [col.replace(' ', '') for col in df.columns]
df.columns = col_names
df.head()

,DstPort,Protocol,Timestamp,FlowDuration,TotFwdPkts,TotBwdPkts,TotLenFwdPkts,TotLenBwdPkts,FwdPktLenMax,FwdPktLenMin,...,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Label,binary_classification
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,...,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign,Benign
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,...,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign,Benign
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,...,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign,Benign
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,...,0.0,0.0,0,0,0.0,0.0,0,0,Benign,Benign
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,...,0.0,0.0,0,0,0.0,0.0,0,0,Benign,Benign


In [9]:
df.columns

Index(['DstPort', 'Protocol', 'Timestamp', 'FlowDuration', 'TotFwdPkts',
       'TotBwdPkts', 'TotLenFwdPkts', 'TotLenBwdPkts', 'FwdPktLenMax',
       'FwdPktLenMin', 'FwdPktLenMean', 'FwdPktLenStd', 'BwdPktLenMax',
       'BwdPktLenMin', 'BwdPktLenMean', 'BwdPktLenStd', 'FlowByts/s',
       'FlowPkts/s', 'FlowIATMean', 'FlowIATStd', 'FlowIATMax', 'FlowIATMin',
       'FwdIATTot', 'FwdIATMean', 'FwdIATStd', 'FwdIATMax', 'FwdIATMin',
       'BwdIATTot', 'BwdIATMean', 'BwdIATStd', 'BwdIATMax', 'BwdIATMin',
       'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags', 'BwdURGFlags',
       'FwdHeaderLen', 'BwdHeaderLen', 'FwdPkts/s', 'BwdPkts/s', 'PktLenMin',
       'PktLenMax', 'PktLenMean', 'PktLenStd', 'PktLenVar', 'FINFlagCnt',
       'SYNFlagCnt', 'RSTFlagCnt', 'PSHFlagCnt', 'ACKFlagCnt', 'URGFlagCnt',
       'CWEFlagCount', 'ECEFlagCnt', 'Down/UpRatio', 'PktSizeAvg',
       'FwdSegSizeAvg', 'BwdSegSizeAvg', 'FwdByts/bAvg', 'FwdPkts/bAvg',
       'FwdBlkRateAvg', 'BwdByts/bAvg', 'BwdPkts/bAvg'

In [10]:
# drop columns
df_feature = df[['FlowDuration', 'TotFwdPkts', 'TotBwdPkts', 'TotLenFwdPkts', 'TotLenBwdPkts', 'FwdPktLenMin', 'FwdPktLenStd', 'BwdPktLenMax', 'BwdPktLenMean'
                 ,'BwdPktLenStd', 'FlowByts/s', 'FlowPkts/s', 'FlowIATStd', 'FwdIATTot', 'FwdIATMean', 'FwdIATMax', 'BwdIATMean', 'BwdIATStd'
                 , 'BwdIATMax', 'BwdIATMin', 'BwdPSHFlags', 'FwdHeaderLen', 'BwdHeaderLen', 'FwdPkts/s', 'BwdPkts/s', 'PktLenMax', 'PktLenStd'
                 , 'FINFlagCnt', 'SYNFlagCnt', 'ACKFlagCnt', 'Down/UpRatio', 'BwdSegSizeAvg', 'FwdHeaderLen', 'SubflowFwdPkts', 'SubflowFwdByts', 'IdleStd', 
                 'SubflowBwdPkts', 'SubflowBwdByts', 'InitBwdWinByts', 'FwdActDataPkts', 'ActiveStd', 'ActiveMax', 'Label', 'binary_classification']]

In [11]:
df_feature

,FlowDuration,TotFwdPkts,TotBwdPkts,TotLenFwdPkts,TotLenBwdPkts,FwdPktLenMin,FwdPktLenStd,BwdPktLenMax,BwdPktLenMean,BwdPktLenStd,...,SubflowFwdByts,IdleStd,SubflowBwdPkts,SubflowBwdByts,InitBwdWinByts,FwdActDataPkts,ActiveStd,ActiveMax,Label,binary_classification
0,112641719,3,0,0,0,0,0.0,0,0.0,0.0,...,0,139.300036,0,0,-1,0,0.0,0,Benign,Benign
1,112641466,3,0,0,0,0,0.0,0,0.0,0.0,...,0,114.551299,0,0,-1,0,0.0,0,Benign,Benign
2,112638623,3,0,0,0,0,0.0,0,0.0,0.0,...,0,301.934596,0,0,-1,0,0.0,0,Benign,Benign
3,6453966,15,10,1239,2273,0,196.741237,976,227.3,371.677892,...,1239,0.0,10,2273,233,6,0.0,0,Benign,Benign
4,8804066,14,11,1143,2209,0,203.745545,976,200.818182,362.249864,...,1143,0.0,11,2209,233,6,0.0,0,Benign,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356594,9487,10,0,0,0.0,0,0.0,0,0.0,0.0,...,0,0.0,0,0,-1,0,0.0,0.0,Bot,Malicious
356709,1030059,4,2,0,0.0,0,0.0,0,0.0,0.0,...,0,0.0,2,0,8192,0,0.0,0.0,Bot,Malicious
356768,1029998,4,2,0,0.0,0,0.0,0,0.0,0.0,...,0,0.0,2,0,8192,0,0.0,0.0,Bot,Malicious
357370,1030017,4,2,0,0.0,0,0.0,0,0.0,0.0,...,0,0.0,2,0,8192,0,0.0,0.0,Bot,Malicious


In [12]:
from sklearn.model_selection import train_test_split

# sample data
label_Benign1_df_feature1, label_Benign2_df_feature1 = train_test_split(df_feature[df_feature['binary_classification'] == 'Benign'], test_size=0.2)
label_Malicious_df_feature, label_Malicious_df_feature2 = train_test_split(df_feature[df_feature['binary_classification'] == 'Malicious'], test_size=0.2)
df = pd.concat([label_Benign2_df_feature1, label_Malicious_df_feature2])

In [13]:
df['binary_classification'].value_counts()

binary_classification
Benign       491286
Malicious    355704
Name: count, dtype: int64

In [14]:
# save the data
df.to_csv('../datasets/target_model/CICIDS2018/target_model_CICIDS2018data.csv', index=False) #datasets\target_model\CICIDS2018